# serverless endpoint

https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html

In [1]:
# !pip3 install sagemaker
#!pip3 install --upgrade botocore

In [2]:
import json

from abcli import fullname
from abcli import string
from roofAI import NAME, VERSION
from roofAI.inference.functions import create_model_and_endpoint

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{NAME}-{VERSION}-{fullname()}, built on {string.pretty_date()}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


roofAI-3.246.1-abcli-7.2554.1-unknown, built on 09 December 2023, 17:29:19


---

## creating the model

In [3]:
model_name = "model-2023-12-03-11-24-39-75649"

In [4]:
import boto3
import sagemaker
region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)

Found credentials in shared credentials file: ~/.aws/credentials


In [5]:
create_model_and_endpoint(
    model_name=model_name,
    verbose=True,
    )

create_model_and_endpoint(model-2023-12-03-11-24-39-75649)


sagemaker.config      - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config      - Not applying SDK defaults from location: /Users/kamangir/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name shamim to get Role path.
sagemaker_role: defaulting to local.
Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.
list_models: {'Models': [{'ModelName': 'model-2023-12-03-11-24-39-75649', 'ModelArn': 'arn:aws:sagemaker:ca-central-1:120429650996:model/model-2023-12-03-11-24-39-75649', 'CreationTime': datetime.datetime(2023, 12, 9, 17, 28, 34, 79000, tzinfo=tzlocal())}], 'ResponseMetadata': {'RequestId': 'c1d057ef-6f69-47b6-ab0e-1128854855e1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c1d057ef-6f69-47b6-ab0e-1128854855e1', 'content-type': 'application/x-amz-json-1.1', 'content-length': '187', 'date': 'Sun, 10 Dec 2023 01:29:20 GMT'}, 'RetryAttempts': 0}}
model already exists, will delete first.
delete_model: {'ResponseMetadata': {'RequestId': '8e15825a-8257-4a5a-90b1-118e522625e0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8e15825a-8257-4a5a-90b1-118e522

True

## Creating the endpoint configuration

In [6]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/list_endpoint_configs.html
response = client.list_endpoint_configs(NameContains=model_name)

endpoint_config_found = bool(response["EndpointConfigs"])
if endpoint_config_found:
    logger.info("endpoint config already exists, will delete first.")

endpoint config already exists, will delete first.


In [7]:
if endpoint_config_found:
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/delete_endpoint_config.html
    response = client.delete_endpoint_config(EndpointConfigName=model_name)
    logger.info(json.dumps(response,indent=4))    

{
    "ResponseMetadata": {
        "RequestId": "5d3854c2-0f91-4de9-b94e-c399c20a4d9d",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "5d3854c2-0f91-4de9-b94e-c399c20a4d9d",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "0",
            "date": "Sun, 10 Dec 2023 01:29:22 GMT"
        },
        "RetryAttempts": 0
    }
}


In [8]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html#serverless-endpoints-create-config

response = client.create_endpoint_config(
   EndpointConfigName=model_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 20,
                #"ProvisionedConcurrency": 10,
            }
        } 
    ]
)

logger.info(json.dumps(response,indent=4))

{
    "EndpointConfigArn": "arn:aws:sagemaker:ca-central-1:120429650996:endpoint-config/model-2023-12-03-11-24-39-75649",
    "ResponseMetadata": {
        "RequestId": "5b5c3dfa-d19c-4320-9158-66e0cab31a5a",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "5b5c3dfa-d19c-4320-9158-66e0cab31a5a",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "115",
            "date": "Sun, 10 Dec 2023 01:29:22 GMT"
        },
        "RetryAttempts": 0
    }
}


In [9]:
response = client.list_endpoint_configs(NameContains=model_name)

if response["EndpointConfigs"]:
    logger.info(response["EndpointConfigs"])
else:
    logger.error("endpoint config was not created.")
    assert False

[{'EndpointConfigName': 'model-2023-12-03-11-24-39-75649', 'EndpointConfigArn': 'arn:aws:sagemaker:ca-central-1:120429650996:endpoint-config/model-2023-12-03-11-24-39-75649', 'CreationTime': datetime.datetime(2023, 12, 9, 17, 29, 23, 277000, tzinfo=tzlocal())}]


---

In [10]:
# END